# Introduction

![metagenomics pipeline image](mg_pipeline.jpg)
The pipeline for analyzing DNA sequences can be broken down into computational steps that makes the process easy to follow. Two different sequence data sets, 16s and shotgun, can be implemented into the ADToolbox for sequence analysis. An example of each will be demonstrated below.

- The first step downloads sequence data from either a SRA sequence database or sequences collected from experimental data; though, if you already have your own sequencing data you can skip this step. 

- The sequence data will then be processed. Depending on the sequencing data (16s or shotgun) you must follow different steps that will be explained below. 

- The final output of each pipeline will be a json file that represents the relative abundance of microbial groups involved in the ADM model. 

Here, we will study two publicly available metagenomics studies, one for 16s & one for shotgun.

- 16s data: https://www.ncbi.nlm.nih.gov/Traces/study/?acc=SRP374594&o=acc_s%3Aa
- shotgun data: https://www.ebi.ac.uk/ena/browser/view/PRJEB34458

## 16s Metagenomics Data

### Step 1: Downloading the 16s samples from SRA

Using the link above, you must download the metadata table from SRA and extract all of the sample accesions and then download the sequence files for each sample using the code below. 
**Note** In this example the name of the metadata file is "SraRunTable.txt". 


In [12]:
import pandas as pd
from adtoolbox import core,configs,utils
import os
import pathlib
import numpy as np
import json

you can download the metadata file using the following code:
```bash
!wget [LINK]
```

In [13]:
sra_metadatatable = pd.read_table("SraRunTable.txt",delimiter=",",header='infer')
dmc = configs.Metagenomics(adtoolbox_docker=configs.ADTOOLBOX_CONTAINERS["docker_arm64"]) # 
mo = core.Metagenomics(dmc) # metadata object
sample_accesions = sra_metadatatable["Run"]
base_seq_dir=pathlib.Path("./16s_samples_sequences/")

In [3]:
for i in sample_accesions:
    os.system(mo.seqs_from_sra(accession=i,target_dir="./16s_samples_sequences/"+i)[0])
# iterating through the sample accesions 

### Step 2: Process the samples with Qiime




In processing the samples with QIIMEII, there are multiple options. If you have QIIMEII installed on your computer, and it exists in the environment, and willing to run QIIME on your computer, you should choose "None" as your container option. If you do not have QIIMEII installed on your computer, you can either use docker or singularity depending on the availability of the software on your computer.

In [1]:

for i in sample_accesions:
    process_samples = mo.run_qiime2_from_sra(
                            read_1 = str(((base_seq_dir/i/i)/(i+"_1.fastq")).absolute()),
                            read_2 = str(((base_seq_dir/i/i)/(i+"_2.fastq")).absolute()),
                            sample_name = i,
                            manifest_dir =str((base_seq_dir/i).absolute()) , #Where the manifest file will be saved
                            workings_dir = str((base_seq_dir/i).absolute()), #Where the intermediate and output files will be saved
                            save_manifest =True , #If you want to run the script use True
                            container = 'docker',
                            p_trunc_len=("240","240"),)
    os.system(process_samples[0])

Now that we have the ASVs, we can align them to GTDB using the following code:
(Note that the docker/singularity container must be selected according to your system and it is a different container than the one used for QIIMEII. The following is for mac with arm processors. Also note that the path to the files must be absolute.)


In [5]:
dmc.adtoolbox_docker=configs.ADTOOLBOX_CONTAINERS["docker_arm64"]
## ALSO you can change the default vsearch similarity threshold by 
dmc.vsearch_similarity=0.95
for i in base_seq_dir.rglob("*/dna-sequences.fasta"):
    os.system(mo.align_to_gtdb(str(i.absolute()),str(i.parent.absolute()),container = 'docker')[0])

Now that we have found our representative genomes we can download them. First, we must extract the genome IDs from the alignment file.

In [3]:
genomes=set() # using sets to avoid duplicates
for i in sample_accesions:
    t=mo.get_genomes_from_gtdb_alignment(str(base_seq_dir/i))
    genomes.update(t.values())

At 0.97 similarity, we get about 450 representative genomes. Let's download them using the following code:

In [7]:
for i in genomes:
    os.system(mo.download_genome(i,(base_seq_dir/"genomes").absolute(),container='docker')[0]) # usually you can run this without a container

Now that we have downloaded the representative genomes, we can align them to the ADToolbox protein database. To do this first we will use a helper method to extract all the genomes that we downloaded and their address:

In [4]:
genomes_info=mo.extract_genome_info(base_dir=(base_seq_dir/"genomes").absolute())

In [5]:
for name,address in genomes_info.items():
    os.system(mo.align_genome_to_protein_db(address=address,outdir=str((base_seq_dir/"genome_alignments").absolute()),name=name,container='docker')[0])

let's extract the ec numbers from the alignment file:

In [5]:
ec_dict={}
for i in (base_seq_dir/"genome_alignments").rglob("Alignment_Results_mmseq*"):
    ec_dict.update({"_".join(i.name.removeprefix("Alignment_Results_mmseq_").split("_")[:2]):mo.extract_ec_from_alignment(str(i.absolute()))})

Now we can convert these ec's to COD contribution of each genome to ADM microbial groups:

In [6]:
cod_dict={}
for i in ec_dict.items():
    cod_dict.update({i[0]:mo.get_cod_from_ec_counts(i[1])})

In the final step we need to find the relative proportion of each microbial group in each sample. To do this we need to first get the feature abundances for each sample:

In [18]:
samples_cod={}
for i in sample_accesions:
    asv_map=mo.get_genomes_from_gtdb_alignment(str(base_seq_dir/i))
    feature_table =pd.read_table((base_seq_dir/i/"feature-table.tsv"),delimiter="\t",skiprows=1)
    feature_table["has_genome"]=feature_table["#OTU ID"].apply(lambda x: x in asv_map.keys())
    feature_table=feature_table[feature_table["has_genome"]]
    feature_table.loc[:,i]=(feature_table[i]/feature_table[i].sum())
    feature_table["genome"]=feature_table["#OTU ID"].apply(lambda x: asv_map[x])
    feature_table=feature_table[["genome",i]]
    temp_cod=pd.DataFrame(np.matmul(pd.DataFrame.from_dict(cod_dict)[feature_table["genome"]].to_numpy(),feature_table[i].to_numpy()),index=pd.DataFrame.from_dict(cod_dict).index)
    temp_cod=temp_cod/temp_cod.sum()
    temp_cod=temp_cod.to_dict()[0]
    samples_cod.update({i:temp_cod})

In [16]:
with open("normalized_16s_samples_cod.json","w") as f:
    json.dump(samples_cod,f)

This is the end of pipeline for 16s data. The final output is a json file that represents the relative abundance of microbial groups involved in the ADM model. This now can be used for modeling work. Check out the parameter tuning notebook for more information on how to use this json file for modeling.

## Shotgun Metagenomics Data

Similar to the previous example, we will download the shotgun data using the script below:

```bash
!wget [LINK]
```

Now let's download the sequence files for each sample using the following code:


In [1]:
import pandas as pd
from adtoolbox import core,configs,utils
import os
import pathlib
import numpy as np
import json
import subprocess
import pandas as pd


In [2]:
sra_metadatatable = pd.read_table("SraRunTable_Shotgun.txt",delimiter="\t")
# dmc = configs.Metagenomics(adtoolbox_docker=configs.ADTOOLBOX_CONTAINERS["docker_arm64"]) # 
dmc = configs.Metagenomics(adtoolbox_docker=configs.ADTOOLBOX_CONTAINERS["docker_arm64"]) # 
mo = core.Metagenomics(dmc) # metadata object
sample_accesions = sra_metadatatable["run_accession"]
base_seq_dir=pathlib.Path("./Shotgun_samples_sequences/")

In [4]:
for sample in sample_accesions:
    os.system(mo.seqs_from_sra(sample,(base_seq_dir/sample).absolute(),container='docker')[0]) # use docker if you don't have SRAToolkit installed


Now that we downloaded the sequence files, we can analyze them. There are two common ways to analyze shotgun data, one is to assemble the short reads which finally yields MAGs with their relative abundance. ADToolbox does not offer assembly functionalitie. However, if you have  your MAGs with their relative abundances, the downstream part will be identicall to when you have 16s data with representative genome:
align genomes to protein db -> extract ec from alignment -> convert ec to cod -> get COD relative abundance for each sample.
However, if you prefer to map the short reads to the protein database directly, you can use an ADToolbox functionality. However, keep in mind that this process is resource intensive usually not possible on a local machine. Fortunately, ADToolbox offers a way to interact with HPCs that work with slurm. Even if you have a very large number of samples you can distribute the tasks in a way that suits your HPC. An example of how to use this functionality is shown below:
###NOTE: These settings are taylored for Alpine HPCs at Colorado State Univeristy. You need to change the settings according to your HPC.

In [5]:
util_conf=configs.Utils(
    slurm_executer="amilan",
    slurm_wall_time="24:00:00",
    slurm_cpus="12",
    slurm_memory="100G")

We have defined a configs object. We have 12 metgenomics samples. We want to run each sample as a task on the HPC (all 12 in parallel). ADtoolbox does not accept paired end metagenomics samples. You have to merge the pared reads into one file. Let's say your files have a layout like this:

 ./Shotgun_samples_sequences/\<SRAACCESSION>/\<SRAACCESSION>_merged.fastq
 
  There are plenty of tools to do this. Also, you can also simply merge the files but this is not recommended. Once you have the samples, you can submit the jobs to the HPC using the following code. Let's see an example for the first task:

In [14]:
sample_addresses=[os.path.join("Shotgun_samples_sequences",i,i+"_merged.fastq") for i in sample_accesions]

In [15]:
commands=utils.generate_batch_script(
        mo.align_short_reads_to_protein_db,
        12,
        input_series=list([sample_addresses,sample_accesions]),
        input_var=["query_seq","alignment_file_name"],
        container='singularity',
        )
for i in sample_accesions:
    print(utils.wrap_for_slurm(command=commands[0],jobname=i,run=False,save=False,config=util_conf))
    break
    

#!/bin/bash
#SBATCH --job-name=ERR3525312
#SBATCH --partition=amilan
#SBATCH --time=24:00:00
#SBATCH --ntasks=12
#SBATCH --mem=100G
#SBATCH --output=ERR3525312.out.log


singularity exec --bind /Users/parsaghadermarzi/Desktop/test_qiime/Shotgun_samples_sequences/ERR3525312:/Users/parsaghadermarzi/Desktop/test_qiime/Shotgun_samples_sequences/ERR3525312 docker://parsaghadermazi/adtoolbox:x86 mmseqs createdb /Users/parsaghadermarzi/Desktop/test_qiime/Shotgun_samples_sequences/ERR3525312/ERR3525312_merged.fastq /Users/parsaghadermarzi/Desktop/test_qiime/Shotgun_samples_sequences/ERR3525312/ERR3525312_merged
singularity exec --bind /Users/parsaghadermarzi/Desktop/test_qiime/Shotgun_samples_sequences/ERR3525312:/Users/parsaghadermarzi/Desktop/test_qiime/Shotgun_samples_sequences/ERR3525312,/Users/parsaghadermarzi/Desktop/ADToolbox/Database:/Users/parsaghadermarzi/Desktop/ADToolbox/Database docker://parsaghadermazi/adtoolbox:x86 mmseqs search /Users/parsaghadermarzi/Desktop/test_qiime/Shotgun

Now we can run the jobs on the HPC using the following code:

In [ ]:
commands=utils.generate_batch_script(
        mo.align_short_reads_to_protein_db,
        12,
        input_series=list([sample_addresses,sample_accesions]),
        input_var=["query_seq","alignment_file_name"],
        container='singularity',
        )
for ind,i in enumerate(sample_accesions):
    utils.wrap_for_slurm(command=commands[ind],jobname=i,run=True,save=False,config=util_conf)
    

Similar to the 16s analysis, we can now extract the ec numbers from the alignment file:
